In [ ]:
# ════════════════════════════════════════════════════════════════════════
# 📦 DEPENDENCIES
# ════════════════════════════════════════════════════════════════════════
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from hydroeval import nse, pbias, rmse
from sklearn.metrics import r2_score
from colorama import Fore, Style, init
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment, Border, Side
from openpyxl.utils import get_column_letter
from typing import Dict, Any, List, TYPE_CHECKING

if TYPE_CHECKING:
    from openpyxl.workbook import Workbook

# Initialize colorama for colored console output
init(autoreset=True)


# ════════════════════════════════════════════════════════════════════════
# ⚙️ USER CONFIGURATION (EDITABLE OPTIONS)
# ════════════════════════════════════════════════════════════════════════

class Config:
    """Holds all user-configurable settings and script constants."""
    # --- Input / Output Paths ---
    BASE_DIR = Path("Your_project_folder")
    OBS_DIR = BASE_DIR / "Observed_monthly_discharge"
    SIM_DIR = BASE_DIR / "Simulated_monthly_discharge"
    COMPARISON_DIR = BASE_DIR / "Discharge_comparison"
    OUTPUT_FILE = "Hydro_Eval_All_Subbasins.xlsx"

    # --- Time Periods for Analysis ---
    CALIBRATION_START = "2000-01-01"
    CALIBRATION_END = "2009-12-31"
    VALIDATION_START = "2010-01-01"
    VALIDATION_END = "2015-12-31"

    # --- Analysis Options ---
    # Set to False to only calculate "Overall" metrics. Set to True to include seasonal breakdowns.
    RUN_SEASONAL_ANALYSIS = False

    # --- Seasonal Definitions (Used only if RUN_SEASONAL_ANALYSIS is True) ---
    SEASONS = {
        'Wet Season': [6, 7, 8, 9, 10],         # June - October
        'Dry Season': [11, 12, 1, 2, 3, 4, 5]    # November - May
    }
    
    # To run specific subbasins, add their IDs (filename without .csv) as strings.
    # Example: ["Subbasin_1", "Subbasin_42"]
    # Leave empty [] to run all subbasins found in the folders.
    SUBBASINS_TO_RUN = []

    # --- Column Names in Source Files ---
    DATE_COL = "Year_Month"
    DATE_FORMAT = "%Y_%b"
    OBS_COL = "OBSERVED"
    SIM_COL = "SIMULATED"

    # --- Metric Calculation Parameters ---
    UNCERTAINTY_PERCENT = 0.5
    STD_TYPE = 1
    NRMSE_MODE = "mean"
    KGE_VERSION = "2009"

    # --- Report Configuration ---
    METRICS_ORDER = [
        "NSE", "R²", "KGE", "PBIAS (%)", "RSR", "RMSE", "MAE", "nRMSE (%)",
        "Pearson r", "Pearson r²", "MAPE (%)", "Bias", "SDR", "P-factor", "R-factor"
    ]

# Instantiate the configuration
cfg = Config()


# ════════════════════════════════════════════════════════════════════════
# 🛠️ HELPER & PRE-PROCESSING FUNCTIONS
# ════════════════════════════════════════════════════════════════════════

def prepare_comparison_files() -> None:
    """
    Merges observed and simulated data files for each subbasin from CSV files.
    """
    print(Fore.CYAN + "\nSTEP 1: Preparing comparison files...")
    cfg.COMPARISON_DIR.mkdir(exist_ok=True)

    obs_files = list(cfg.OBS_DIR.glob("*.csv"))
    sim_files_map = {f.stem: f for f in cfg.SIM_DIR.glob("*.csv")}

    if not obs_files:
        raise FileNotFoundError(f"No observed data files (.csv) found in {cfg.OBS_DIR}")

    for obs_path in tqdm(obs_files, desc="🤝 Merging Obs/Sim Files"):
        subbasin_id = obs_path.stem
        if subbasin_id in sim_files_map:
            sim_path = sim_files_map[subbasin_id]
            try:
                df_obs = pd.read_csv(obs_path)
                df_sim = pd.read_csv(sim_path)
                df_obs['Date'] = pd.to_datetime(df_obs[cfg.DATE_COL], format=cfg.DATE_FORMAT)
                df_sim['Date'] = pd.to_datetime(df_sim[cfg.DATE_COL], format=cfg.DATE_FORMAT)
                df_merged = pd.merge(df_obs, df_sim, on='Date', how='inner')
                output_df = df_merged[['Date', cfg.OBS_COL, cfg.SIM_COL]].copy()
                output_df.rename(columns={'Date': cfg.DATE_COL}, inplace=True)
                output_df[cfg.DATE_COL] = output_df[cfg.DATE_COL].dt.strftime(cfg.DATE_FORMAT)
                output_path = cfg.COMPARISON_DIR / f"{subbasin_id}.csv"
                output_df.to_csv(output_path, index=False)
            except Exception as e:
                print(Fore.RED + f"\n  - Error processing {subbasin_id}: {e}")
        else:
            print(Fore.YELLOW + f"\n  - Warning: No matching simulated file for {subbasin_id}")

def safe_divide(numerator: float, denominator: float, default: Any = np.nan) -> float:
    """
    Performs division, returning a default value if the denominator is zero or invalid.
    """
    if denominator is None or denominator == 0 or np.isnan(denominator):
        return default
    return numerator / denominator


# ════════════════════════════════════════════════════════════════════════
# 📊 METRIC COMPUTATION
# ════════════════════════════════════════════════════════════════════════

def compute_metrics(obs_series: pd.Series, sim_series: pd.Series) -> Dict[str, Any]:
    """
    Computes a comprehensive set of hydrological performance metrics.
    """
    obs_arr = np.asarray(obs_series).ravel()
    sim_arr = np.asarray(sim_series).ravel()
    mask = ~np.isnan(obs_arr) & ~np.isnan(sim_arr)
    obs, sim = obs_arr[mask], sim_arr[mask]

    if obs.size < 2:
        return {m: np.nan for m in cfg.METRICS_ORDER}

    mean_obs, std_obs = obs.mean(), obs.std(ddof=cfg.STD_TYPE)
    mean_sim, std_sim = sim.mean(), sim.std(ddof=cfg.STD_TYPE)
    diff = sim - obs
    rmse_val = np.sqrt(np.mean(diff**2))
    pearson_r = np.corrcoef(obs, sim)[0, 1]
    nrmse_normalizers = {"mean": mean_obs, "range": obs.max() - obs.min(), "std": std_obs}
    nrmse_val = safe_divide(rmse_val, nrmse_normalizers.get(cfg.NRMSE_MODE, mean_obs)) * 100

    if cfg.KGE_VERSION == "2012":
        beta = safe_divide(mean_sim, mean_obs)
        gamma = safe_divide(safe_divide(std_sim, mean_sim), safe_divide(std_obs, mean_obs))
        kge = 1 - np.sqrt((pearson_r - 1)**2 + (beta - 1)**2 + (gamma - 1)**2)
    else:
        kge = 1 - np.sqrt((pearson_r - 1)**2 + (safe_divide(mean_sim, mean_obs) - 1)**2 + (safe_divide(std_sim, std_obs) - 1)**2)

    delta = np.abs(sim) * cfg.UNCERTAINTY_PERCENT
    p_factor = np.sum((obs >= (sim - delta)) & (obs <= (sim + delta))) / len(obs) if len(obs) > 0 else np.nan
    r_factor = safe_divide(np.mean(2 * delta), std_obs)
    mape = np.nanmean(np.abs(diff / np.where(obs == 0, np.nan, obs))) * 100

    return {
        "NSE": nse(sim, obs), "R²": r2_score(obs, sim), "Pearson r": pearson_r,
        "Pearson r²": pearson_r**2, "RSR": safe_divide(rmse_val, std_obs),
        "nRMSE (%)": nrmse_val, "PBIAS (%)": pbias(sim, obs), "KGE": kge,
        "RMSE": rmse_val, "MAE": np.mean(np.abs(diff)), "MAPE (%)": mape,
        "Bias": diff.mean(), "SDR": safe_divide(std_sim, std_obs),
        "P-factor": p_factor, "R-factor": r_factor
    }


# ════════════════════════════════════════════════════════════════════════
# 📑 DATA PROCESSING & EXCEL FORMATTING
# ════════════════════════════════════════════════════════════════════════

def tag_season(df: pd.DataFrame) -> pd.DataFrame:
    """Adds 'Month' and 'Season' columns to a DataFrame based on the date."""
    df["Month"] = df["Date"].dt.month
    month_to_season = {m: s for s, months in cfg.SEASONS.items() for m in months}
    df["Season"] = df["Month"].map(month_to_season)
    return df

def style_workbook(wb: "Workbook") -> None:
    """Applies consistent styling to all sheets in the final Excel workbook."""
    border = Border(left=Side(style="thin"), right=Side(style="thin"), top=Side(style="thin"), bottom=Side(style="thin"))
    header_font = Font(bold=True)
    center_align = Alignment(horizontal="center", vertical="center", wrap_text=True)
    left_align = Alignment(horizontal="left", vertical="center", wrap_text=True)

    for ws in wb.worksheets:
        # Style header row (e.g., Period, Subbasin, NSE, R², etc.)
        for cell in ws[1]:
            cell.font = header_font
            cell.alignment = center_align
            cell.border = border

        # Style data rows
        for row in ws.iter_rows(min_row=2):
            for i, cell in enumerate(row):
                cell.border = border
                # Left-align the first two columns (Period, Subbasin index)
                if i < 2:
                    cell.alignment = left_align
                # Center-align the rest (metric values)
                else:
                    cell.alignment = center_align

        # Auto-fit all column widths
        for col_idx in range(1, ws.max_column + 1):
            col_letter = get_column_letter(col_idx)
            try:
                max_len = max(len(str(cell.value)) for cell in ws[col_letter] if cell.value)
                ws.column_dimensions[col_letter].width = max_len + 5
            except (ValueError, TypeError):
                ws.column_dimensions[col_letter].width = 15

# ════════════════════════════════════════════════════════════════════════
# 🔁 MAIN EVALUATION SCRIPT
# ════════════════════════════════════════════════════════════════════════

def main() -> None:
    """
    Main script: prepares data, computes metrics for Cal/Val, and saves report.
    """
    prepare_comparison_files()

    print(Fore.CYAN + "\nSTEP 2: Starting Hydrological Performance Evaluation...")
    input_files = [f for f in cfg.COMPARISON_DIR.glob("*.csv") if not f.name.startswith('~$')]

    # --- Filter for specific subbasins if requested by the user ---
    if cfg.SUBBASINS_TO_RUN:
        print(Fore.YELLOW + f"Filtering for specific subbasins: {cfg.SUBBASINS_TO_RUN}")
        input_files = [f for f in input_files if f.stem in cfg.SUBBASINS_TO_RUN]

    if not input_files:
        print(Fore.RED + f"⚠️ No files to process. Check SUBBASINS_TO_RUN or the contents of: {cfg.COMPARISON_DIR}. Exiting.")
        return

    all_results = []
    for file_path in tqdm(input_files, desc="📊 Evaluating Subbasins"):
        subbasin = file_path.stem
        try:
            df = pd.read_csv(file_path)
            df[cfg.OBS_COL] = pd.to_numeric(df[cfg.OBS_COL], errors='coerce')
            df[cfg.SIM_COL] = pd.to_numeric(df[cfg.SIM_COL], errors='coerce')
            df['Date'] = pd.to_datetime(df[cfg.DATE_COL], format=cfg.DATE_FORMAT, errors='coerce')
            df.dropna(subset=[cfg.OBS_COL, cfg.SIM_COL, 'Date'], inplace=True)
            df = tag_season(df)

            periods = {
                "Calibration": (df['Date'] >= cfg.CALIBRATION_START) & (df['Date'] <= cfg.CALIBRATION_END),
                "Validation": (df['Date'] >= cfg.VALIDATION_START) & (df['Date'] <= cfg.VALIDATION_END)
            }

            for eval_period_name, date_mask in periods.items():
                period_df = df[date_mask]
                if period_df.empty: continue

                # --- Calculate metrics for Overall and optionally Seasonal ---
                groups = {"Overall": period_df}
                if cfg.RUN_SEASONAL_ANALYSIS:
                    groups.update({name: data for name, data in period_df.groupby("Season")})

                for period_name, group_df in groups.items():
                    if group_df.empty: continue
                    metrics = compute_metrics(group_df[cfg.OBS_COL], group_df[cfg.SIM_COL])
                    for metric_name, value in metrics.items():
                        all_results.append({
                            "EvaluationPeriod": eval_period_name, "Subbasin": subbasin,
                            "Period": period_name, "Metric": metric_name, "Value": value
                        })
        except Exception as e:
            print(Fore.RED + f"  - Error processing {file_path.name}: {e}")

    if not all_results:
        print(Fore.YELLOW + "\nNo valid data found to generate a report. Exiting.")
        return

    print(Fore.CYAN + "\nSTEP 3: Generating final Excel report...")
    results_df = pd.DataFrame(all_results)
    output_path = cfg.BASE_DIR / cfg.OUTPUT_FILE

    try:
        with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
            season_order = ["Overall"] + list(cfg.SEASONS.keys())

            for eval_period in ["Calibration", "Validation"]:
                eval_df = results_df[results_df['EvaluationPeriod'] == eval_period]
                if eval_df.empty: continue

                # --- NEW: Logic for Natural Sorting of Subbasins ---
                unique_subbasins = eval_df['Subbasin'].unique()
                def sort_key(subbasin_name):
                    try:
                        return int(subbasin_name.split('_')[-1]) # Assumes "Name_Number" format
                    except (ValueError, IndexError):
                        return subbasin_name # Fallback for other formats
                sorted_subbasins = sorted(unique_subbasins, key=sort_key)
                # --- End of New Sorting Logic ---

                pivot = eval_df.pivot_table(
                    index=["Period", "Subbasin"],
                    columns="Metric",
                    values="Value"
                )

                if not pivot.empty:
                    # Ensure columns (Metrics) are in the desired order
                    pivot = pivot.reindex(columns=cfg.METRICS_ORDER)
                    # Ensure the 'Period' level of the index is in the desired order
                    pivot = pivot.reindex(level='Period', index=season_order)
                    # --- NEW: Apply the natural sort order to the 'Subbasin' index level ---
                    pivot = pivot.reindex(level='Subbasin', index=sorted_subbasins)
                    pivot = pivot.round(3)

                pivot.to_excel(writer, sheet_name=eval_period)

            style_workbook(writer.book)
        print(Fore.GREEN + Style.BRIGHT + f"\n✅ Processing complete! Results saved to: {output_path}")
    except Exception as e:
        print(Fore.RED + f"\n❌ Failed to write Excel file: {e}")


# ════════════════════════════════════════════════════════════════════════
# ▶️ SCRIPT EXECUTION
# ════════════════════════════════════════════════════════════════════════

if __name__ == "__main__":
    main()